In [38]:
import Embedding_Model as em
import os
import zipfile
import shutil


import Embedding_Model as em
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
from sklearn.decomposition import PCA

<h3>Extracting tender documents from zip files<h3>

In [2]:
# extract tenders from zip files using mitch's code 

def search(search_path):

    ref_dict = {}

# recursively through all files and folders
    for root, dirs, files in os.walk(search_path):
        for filename in files:
            if filename.endswith(".zip"):
                # get ref num
                ref = filename.rsplit("-specification.zip", 1)[0]
                file_path = os.path.join(root, filename)

                # open read zip
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    file_list = zip_ref.namelist()
                    doc_files = [file for file in file_list if "request" in file.lower() and file.lower().endswith(('.doc', '.docx'))]

                    if doc_files:
                        # add sub files, if any
                        ref_dict.setdefault(ref, []).extend([(doc_name, file_path) for doc_name in doc_files])

    for ref, doc_list in ref_dict.items():
        print(f"Reference: {ref}")
        for doc_name, file_path in doc_list:
            print(f"Document Name: {doc_name}, ZIP File Path: {file_path}")

    return ref_dict

###########################################
### filter for one relevant doc per ref ###
###########################################

def copy(copy_path, ref_dict):
    # make output folder, in not already there
    os.makedirs(copy_path, exist_ok=True)

    for ref, doc_list in ref_dict.items():
        for doc_name, file_path in doc_list:
            # copy relevant docs from zip to output folder
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                for file in zip_ref.namelist():
                    if file == doc_name:
                        extracted_path = os.path.join(copy_path, f"{ref}.docx")
                        with zip_ref.open(file) as source, open(extracted_path, "wb") as target:
                            shutil.copyfileobj(source, target)
                        print(f"Extracted: {doc_name} from {file_path} to {extracted_path}")
    return

zip_files_path = "..\..\data\Tenders\Tenders"


output_folder = r"..\..\data\tender_docs_extracted_"

def extract(search_path, copy_path):
    ref_dict = search(search_path)
    copy(copy_path, ref_dict)    
    return

extract(zip_files_path, output_folder)

Reference: 02022021DLGSCPTT
Document Name: 02022021DLGSCPTT Request.docx, ZIP File Path: ..\..\data\Tenders\Tenders\02022021DLGSCPTT-specification.zip
Reference: 18044848001
Document Name: ADDENDUM 1 TO REQUEST DOCUMENT.DOC, ZIP File Path: ..\..\data\Tenders\Tenders\18044848001-specification.zip
Reference: 18086197002
Document Name: eDoc - Request For Tender V 3.docx, ZIP File Path: ..\..\data\Tenders\Tenders\18086197002-specification.zip
Reference: 18122020DLGSCPTT
Document Name: Request #18122020DLGSCPTT - Cleaning Services - AEC - Final.DOCX, ZIP File Path: ..\..\data\Tenders\Tenders\18122020DLGSCPTT-specification.zip
Reference: 19059841006
Document Name: Request P010 Fire and Smoke Wall Compartments Construction.DOC, ZIP File Path: ..\..\data\Tenders\Tenders\19059841006-specification.zip
Reference: 19064022
Document Name: ADDENDUM TO REQUEST DOCUMENT.DOC, ZIP File Path: ..\..\data\Tenders\Tenders\19064022-specification.zip
Reference: 20066768001
Document Name: Request - Works and M

PermissionError: [Errno 13] Permission denied: '..\\..\\data\\tender_docs_extracted_\\WR05042022ACT.docx'

<h3>Extracting content from word files <h3>

In [ ]:
# extractign content using enam and max's code 
import PyPDF2
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_number in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_number]
            text += page.extract_text()
    return text


In [ ]:

import re

def normalize_heading(heading):
    # Remove extra spaces and other whitespace characters from the heading
    return re.sub(r'\s+', ' ', heading.strip())

import pdfplumber

def extract_section(contents, heading):
    section_contents = ""
    take_content = False
    for line in contents:
        line = line.strip()
        if heading in line:
            take_content = True
        elif re.match("[0-9]\. *", line):
            take_content = False
            
        if take_content:
            section_contents += line + "\n"
    return section_contents

def extract_content_by_headings(target_headings, pdf_path):
    content = ""
    with pdfplumber.open(pdf_path) as pdf:
        normalized_headings = [normalize_heading(heading) for heading in target_headings]
        is_target_heading = False
        skip_content = False

        contents = []
        for page in pdf.pages:
            contents.extend(page.extract_text().split('\n'))
        
        for heading in normalized_headings:
            content += extract_section(contents, heading)

    return content.strip()

headings_to_extract = ['INTRODUCTION', 'BACKGROUND']

In [ ]:
headings_to_extract = ['INTRODUCTION', 'BACKGROUND']
documents = []
for item in os.listdir(r"..\..\data\tender_docs_extracted_"):
    link = output_folder + "\\" +item
    extract_content_by_headings(headings_to_extract, link)

In [19]:
from docx import Document

def extract_doc_by_header(docx_path, target_header):
    doc = Document(docx_path)
    content = []
    current_header = None
    capture_content = False

    for paragraph in doc.paragraphs:
        if paragraph.style.name.startswith("Heading") and paragraph.text == target_header:
            current_header = paragraph.text
            capture_content = True
            content.append(current_header)
        elif capture_content:
            content.append(paragraph.text)
        
        # Stop capturing content if a new header is encountered
        if paragraph.style.name.startswith("Heading") and paragraph.text != current_header:
            capture_content = False
   
    return "\n".join(content)



In [20]:
header = "Background"
documents = []
failed =0

for item in os.listdir(r"..\..\data\tender_docs_extracted_"):
    try:
        
        link = output_folder + "\\" +item
       
        documents.append(extract_doc_by_header(link,header))
    except:
        failed+=1
        
        continue

In [36]:
documents

clipped_documents = []
for item in documents:
    thing = item[11::]
    thing = thing[:-20:]
    clipped_documents.append(thing)


In [43]:
# initialize model and preprocesser 
model = em.Pretrained_model('bert')
preprocessor = em.Textprocessor(model="bert")
# get embeddings
output = []
c = 0
for item in clipped_documents:
    if len(item) > 0:
        output.append(em.preprocess_and_forward(item,preprocessor,model))
    else: 
        continue
        


In [46]:
# save numpy file
np.savez("document_embeddings.npz",*output)

In [47]:
# restore numpy file
a = np.load("document_embeddings.npz")
loaded_array_list = [a[f] for f in a.files]
loaded_array_list